In [4]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from pyproj import Transformer
from shapely.ops import transform
import numpy as np

# 1. Configuração Inicial
CRS_GEOGRAFICO = 'EPSG:4326'  # WGS84
CRS_PROJETADO = 'EPSG:31983'  # SIRGAS 2000 / UTM zone 23S
LIMITE_DISTANCIA_METROS = 500

# 2. Funções Auxiliares Atualizadas
def setup_transformer():
    """Configura o transformer para conversão de coordenadas"""
    return Transformer.from_crs(
        CRS_GEOGRAFICO,
        CRS_PROJETADO,
        always_xy=True
    ).transform

def calcular_distancia_metros(geom1, geom2, transformer):
    """Calcula distância em metros entre geometrias usando CRS projetado"""
    geom1_proj = transform(transformer, geom1)
    geom2_proj = transform(transformer, geom2)
    return geom1_proj.distance(geom2_proj)

def carregar_dados():
    """Carrega e prepara os dados geográficos"""
    try:
        # Carregar dados com validação
        ruas = gpd.read_file("ruas_baixada.geojson").to_crs(CRS_GEOGRAFICO)
        ciclovias = gpd.read_file("ciclo_baixada.geojson").to_crs(CRS_GEOGRAFICO)
        pois = gpd.read_file("pontos_baixada.geojson").to_crs(CRS_GEOGRAFICO)

        # Converter POIs para pontos de forma segura
        pois['geometry'] = pois.geometry.apply(
            lambda geom: geom.centroid if geom and not geom.is_empty and geom.geom_type != 'Point' else geom
        )

        # Remover geometrias inválidas
        pois = pois[~pois.is_empty]
        ciclovias = ciclovias[~ciclovias.is_empty]

        return ruas, ciclovias, pois
    except Exception as e:
        print(f"Erro ao carregar dados: {str(e)}")
        raise

# 3. Processamento Principal Otimizado
def analisar_ciclovias():
    # Carregar dados
    ruas, ciclovias, pois = carregar_dados()
    transformer = setup_transformer()

    # Pré-processar ciclovias projetadas
    ciclovias_proj = ciclovias.geometry.apply(lambda geom: transform(transformer, geom))

    # Calcular distâncias de forma vetorizada
    def calcular_distancia_mais_proxima(poi):
        poi_proj = transform(transformer, poi)
        distancias = ciclovias_proj.apply(lambda cv: cv.distance(poi_proj))
        return distancias.min()

    pois['distancia_ciclovia'] = pois.geometry.apply(calcular_distancia_mais_proxima)
    pois['distancia_ciclovia'] = pois['distancia_ciclovia'].round(2)

    # Filtrar POIs distantes
    pois_longe_ciclovia = pois[pois['distancia_ciclovia'] > LIMITE_DISTANCIA_METROS].copy()

    return ciclovias, pois, pois_longe_ciclovia

# 4. Visualização (mantida igual)
def criar_mapa_interativo(ciclovias, pois, pois_longe_ciclovia):
    """Cria mapa Folium com visualização dos dados"""
    mapa = folium.Map(location=[-23.9280, -46.3924], zoom_start=12, tiles='CartoDB positron')

    # Adicionar camada de ciclovias
    folium.GeoJson(
        ciclovias,
        name='Ciclovias',
        style_function=lambda x: {'color': '#1E90FF', 'weight': 4},
        tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['Ciclovia:'])
    ).add_to(mapa)

    # Adicionar POIs
    for _, row in pois.iterrows():
        color = '#E74C3C' if row['distancia_ciclovia'] > LIMITE_DISTANCIA_METROS else '#2ECC71'
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5 if color == '#2ECC71' else 7,
            color=color,
            fill=True,
            fill_color=color,
            popup=f"POI: {row.get('name', 'N/A')}<br>Distância: {row['distancia_ciclovia']:.0f}m",
        ).add_to(mapa)

    folium.LayerControl().add_to(mapa)
    return mapa

# 5. Execução principal
def main():
    print("Iniciando análise de ciclovias...")

    try:
        ciclovias, pois, pois_longe_ciclovia = analisar_ciclovias()
        mapa = criar_mapa_interativo(ciclovias, pois, pois_longe_ciclovia)

        mapa.save("mapa_ciclovias_baixada_santista.html")
        pois_longe_ciclovia.to_file("pois_distantes_ciclovias.geojson", driver='GeoJSON')

        print(f"Análise concluída. {len(pois_longe_ciclovia)} POIs distantes das ciclovias.")
    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")

if __name__ == "__main__":
    main()

Iniciando análise de ciclovias...
Análise concluída. 176 POIs distantes das ciclovias.
